Lectura y preparacion de Datos.

In [1]:
#Libraries
import math
from datetime import date, datetime
from itertools import repeat
import numpy as np
import pandas as pd
from tabulate import tabulate
#Intern Requirements
from parametros import *

Comenzamos!

In [3]:
dfcups = pd.read_excel("exceles/CurvaCuartoHoraria_VariosContratos.xlsx")
dfcups.head()

,CUPS,FechaMedida,EPOCA,ActivaEntrante,ActivaSaliente,ReactivaQ1,ReactivaQ2,ReactivaQ3,ReactivaQ4
0,ES0031300839367001RL,2021-10-22 00:15:00,1,54,0,5,0,0,1
1,ES0031300839367001RL,2021-10-22 00:30:00,1,48,0,4,0,0,0
2,ES0031300839367001RL,2021-10-22 00:45:00,1,50,0,8,0,0,0
3,ES0031300839367001RL,2021-10-22 01:00:00,1,42,0,2,0,0,0
4,ES0031300839367001RL,2021-10-22 01:15:00,1,57,0,7,0,0,0


In [4]:
print('Tenemos ' + str(dfcups['CUPS'].nunique()) + ' CUPS distintos:')
print()
print(dfcups['CUPS'].value_counts())

lista_cups = list(dfcups['CUPS'].unique())

Tenemos 8 CUPS distintos:

ES0031405509076001WC    37056
ES0031405835487001AB    37056
ES0031405933271001TV    37056
ES0031406126350001SC    37056
ES0031406126362001PZ    37056
ES0031446444218001WB    37056
ES0031406134794001CF     9988
ES0031300839367001RL     2980
Name: CUPS, dtype: int64


Mask 1

In [5]:
mask1 = dfcups['CUPS'].str.contains(lista_cups[-4])
cups_n1 = dfcups[mask1]
cups_n1 = cups_n1.reset_index(drop = True)
print('CUPS:' + str(lista_cups[-4]))
print('Nº Registros: ' + str(len(cups_n1.index)))
cups_n1.head(5)

CUPS:ES0031406126350001SC
Nº Registros: 37056


,CUPS,FechaMedida,EPOCA,ActivaEntrante,ActivaSaliente,ReactivaQ1,ReactivaQ2,ReactivaQ3,ReactivaQ4
0,ES0031406126350001SC,2020-11-01 00:15:00,0,11,0,1,0,0,1
1,ES0031406126350001SC,2020-11-01 00:30:00,0,12,0,0,0,0,0
2,ES0031406126350001SC,2020-11-01 00:45:00,0,10,0,1,0,0,1
3,ES0031406126350001SC,2020-11-01 01:00:00,0,8,0,1,0,0,0
4,ES0031406126350001SC,2020-11-01 01:15:00,0,6,0,0,0,0,1


In [6]:
cups_n1.ActivaEntrante.max()

43

Transformación

In [7]:
#Try using .loc[row_indexer,col_indexer] = value instead
cups_n1['Fecha'] = pd.to_datetime(cups_n1['FechaMedida']).dt.date
cups_n1['Hora'] = pd.to_datetime(cups_n1['FechaMedida']).dt.time
cups_n1['Mes'] = pd.to_datetime(cups_n1['FechaMedida']).dt.month_name(locale= 'ES')
cups_n1['Dia'] = pd.to_datetime(cups_n1['FechaMedida']).dt.day_name(locale= 'ES')

cups_n1.drop(['FechaMedida'], axis = 1, inplace = True) 

cups_n1['Fecha'] = cups_n1['Fecha'].astype('string') 
cups_n1['Hora'] = cups_n1['Hora'].astype('string') 
cups_n1['Mes'] = cups_n1['Mes'].astype('string') 
cups_n1['Dia'] = cups_n1['Dia'].astype('string') 
#cups_n1.head(5)

cups_n1.dtypes

CUPS              object
EPOCA              int64
ActivaEntrante     int64
ActivaSaliente     int64
ReactivaQ1         int64
ReactivaQ2         int64
ReactivaQ3         int64
ReactivaQ4         int64
Fecha             string
Hora              string
Mes               string
Dia               string
dtype: object

P1,P2,P3,P4,P5,P6

In [8]:
periodo = []

#Apaño momentaneo, no soy capaz de trabajar con Objects /Solucionado -> .item()
for registro in range(len(cups_n1.index)):
    mascara1 = str(cups_n1['Mes'][registro])
    mascara2 = str(cups_n1['Hora'][registro]) #[1:]
    result = tabla_periodos[tabla_periodos['Mes'] == mascara1][mascara2].item()
    periodo.append(result)

cups_n1['Periodo'] = periodo

Festivos Nacionales

In [9]:
festivos = []

for fecha in festivos_nacionales:
    #prueba = festivos_nacionales[1]
    solucion = datetime.strptime(fecha, '%d/%m/%Y').strftime('%Y-%m-%d')
    festivos.append(solucion)

print(festivos)

['2019-01-01', '2019-04-19', '2019-05-01', '2019-08-15', '2019-10-12', '2019-11-01', '2019-12-06', '2019-12-25', '2020-01-01', '2020-01-06', '2020-04-10', '2020-01-01', '2020-05-01', '2020-08-15', '2020-10-12', '2020-12-08', '2020-12-25', '2021-01-01', '2021-01-06', '2021-04-02', '2021-05-01', '2021-10-12', '2021-11-01', '2021-12-06', '2021-12-08', '2021-12-25', '2022-01-01', '2022-01-06', '2022-04-15', '2022-08-15', '2022-10-12', '2022-11-01', '2022-12-06', '2022-12-08']


Aplicamos para Sabados, Domingos y Feestivo el Periodo correspondiente = P6

In [10]:
for i, celda in enumerate(cups_n1.Dia):
    if celda == 'Sabado' or celda == 'Domingo':
        cups_n1.Periodo[i] = 'P6'

for i, celda in enumerate(cups_n1.Fecha):
    if celda  in festivos:
        cups_n1.Periodo[i] = 'P6'

c:\Users\Usuario\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\Users\Usuario\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [11]:
#Dividir la factura en Meses
#Buscar los meses sobre los que tenemos información / Ej: '2020-11$
list_months = cups_n1.Fecha.unique()
list_uniq_months = [x[:-3] for x in list_months]
set_unique_dates = set(list_uniq_months)
list_unique_dates = list(set_unique_dates)
list_unique_dates.sort()

#IMP: list_unique_dates

#Formulas

Para los puntos de medida tipo 4 y 5 con excesos, se seguirá facturando mediante maxímetro, estableciéndose el doble del Término de Exceso de Potencia, para el excedente de la potencia demandada.

Para el resto de los puntos de medida, tipo 1, 2 y 3, se mantiene el procedimiento de facturación mediante curva de carga cuartohoraria, donde el Término de Exceso de Potencia que sea de aplicación tendrá un recargo para desincentivar la contratación de potencias inferiores a las demandadas.

- Tipo 1: los que tienen una potencia contratada sea igual o superior a 10 MW, 12 MVA o 5 GWh.
- Tipo 2: los que soportan una potencia igual o superior a 450 kW, 450 kVA o 750 MWh.
- Tipo 3: los que no se pueden establecer en ninguna de las categorías.
- Tipo 4: se instalan en puntos en los que la potencia contratada es igual o inferior a 50 kW o 50 kVA y superior a 15 kW y 15 kVA.
- Tipo 5: los clientes han fijado una potencia igual o inferior a 15 kW o a 15 kVA.

In [13]:
def precio_exceso_potencia(curva, tarifa, tipo_punto):
    if np.any(tipo_punto <= 3):
        price = tbl_excPot_med123[tbl_excPot_med123['Tarifa'] == tarifa]['Precio_exceso_potencia_(€/kW*día)'].item()
    else:
        price = tbl_excPot_med4y5[tbl_excPot_med4y5['Tarifa'] == tarifa]['Precio_exceso_potencia_(€/kW*día)'].item()
    
    curva['exceso_pot'] = [float(price)] * len(curva.index)

In [14]:
def precio_coeficiente_Kp(curva, tarifa):
    inlist = []
    for period in curva['Periodo']:
        coef_kp = tbl_coeficiente_Kp[tbl_coeficiente_Kp['Tarifa'] == tarifa][period].item()
        inlist.append(coef_kp)
    curva['precio_Kp'] = inlist

In [19]:
tbl_termPrecPot.columns = ['Tarifa', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6']

def facturacion_potencia(curva, tarifa):
    inlist = []
    for period in curva['Periodo']:
        item = tbl_termPrecPot[tbl_termPrecPot['Tarifa'] == tarifa][period].item()
        inlist.append(item)
    curva['fact_pot'] = inlist

In [16]:
potC = [8, 10, 13, 14, 15, 15]

def potencia_contratada(value, potC):
    if value == 'P1':
        return potC[0]
    elif value == 'P2':
        return potC[1]
    elif value == 'P3':
        return potC[2]
    elif value == 'P4':
        return potC[3]
    elif value == 'P5':
        return potC[4]
    elif value == 'P6':
        return potC[5]

![Formula FEP=facturación de los excesos de potencia ](pngs/formula3.png)

- Pdj = Potencia demandada en cada uno de los cuartos de hora j del periodo p, en el que se ha sobrepasado Pcp.

- Pcp = Potencia contratada en el periodo p considerado.

- tep = Termino_exceso_potencia (**). Los precios del término del exceso de potencia (€/kW) de aplicación hasta el 31 de diciembre
      de 2022, a los consumidores a los que les resulte de aplicación el método de facturación establecido en el 
      artículo 9.4.b).2 de la Circular 3/2020 (puntos de suministro con tipo medida 1, 2 y 3).

- Kp = Es un parámetro que toma valores distintos en función del peaje al que se encuentre acogido el cliente y 
     del período sobre el que se aplica. Será calculado y publicado anualmente por la CNMC. Nunca es un cálculo 
     que se realice por cliente.


1ª Opción)

Tenemos que realizar el cálculo distinguinedo:
- Mes a Facturar
- Periodo

In [ ]:
precio_exceso_potencia(cups_n1, '6_3TD', 1)
precio_coeficiente_Kp(cups_n1, '6_3TD')
facturacion_potencia(cups_n1, '6_3TD')

In [ ]:
cups_n1.head(2)

Esto tiene que estar metido dentro de la optimización!!!!

In [ ]:
cups_n1['pot_Contratada'] = cups_n1.apply(lambda x: potencia_contratada(x['Periodo'], potC), axis=1)

In [ ]:
'''para el mes XX/202X
     durante los periodos: P1,P2,P3,P4,P5,P6
     SUM(). kp(tarifa, periodo) * tep() * sqrt(SUM(). Diff(pot - potC)**2'''


new_data = pd.DataFrame(columns=['Month','P1','P2','P3','P4','P5','P6', 'Total_amount'])

for date in list_unique_dates:
    monthly_payment = [date]
    mask_1 = cups_n1["Fecha"].str.startswith(date)
    for periodo in periodos:
        mask_2 = cups_n1['Periodo'] == periodo
        if cups_n1[(mask_1) & (mask_2)].empty == True:
            monthly_payment.append(0)
        else: 
            #1
            piece = cups_n1[(mask_1) & (mask_2)].ActivaEntrante - cups_n1[(mask_1) & (mask_2)].pot_Contratada
            #2
            piece.clip(lower = 0, inplace=True)
            #3
            piece.apply(lambda x: x**2)
            #4
            precio_coeficiente_Kp = cups_n1[(mask_1) & (mask_2)].precio_Kp.unique().item()
            tep = cups_n1[(mask_1) & (mask_2)].exceso_pot.unique().item()
            #5
            fep = precio_coeficiente_Kp * tep * math.sqrt(piece.sum())
            #6
            monthly_payment.append(round(fep, 3))
    #7    
    total_amount = sum(filter(lambda i: isinstance(i, (int, float)), monthly_payment))   
    #8
    monthly_payment.append(round(total_amount, 3))
    #print(monthly_payment)
    new_data.loc[len(new_data)] = monthly_payment

#Print 
print(tabulate(new_data,  headers="keys", showindex=True, tablefmt='fancy_grid'))

2ª Opción)

In [ ]:
potC = [8, 10, 13, 14, 15, 15]
tarifa = '6_3TD'
tipo_punto = 3

precio_fijo_potencia = facturacion_potencia(potC, tarifa)
tep = precio_exceso_potencia(tarifa, tipo_punto)
periodos = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6']
mesuario = {}

for date in list_unique_dates:
    list_mes = []

    for period in periodos:
        new_df = cups_n1[(cups_n1["Fecha"].str.startswith(date)) & (cups_n1['periodo'] == period)]
        new_df.reset_index(drop=True, inplace=True)
        list_exceso = []

        for i, celda in enumerate(new_df.ActivaEntrante):

            num_PX = int(period[1:])
            pot_contratada = potC[(num_PX-1)]

            if celda > pot_contratada:
                result_123 = (celda - pot_contratada) ** 2
            elif celda < pot_contratada or celda == pot_contratada:
                result_123 = 0
            
            #list_SinExceso.append(result_123)
            list_exceso.append(result_123)

        sumatorio_1 = sum(list_exceso)
        #1Formula
        fep = precio_coeficiente_Kp(tarifa, period) * tep * math.sqrt(sumatorio_1)
        list_mes.append(fep)

    precio_mes = round(int(sum(list_mes)), 2)
    mesuario[date] = precio_mes, precio_fijo_potencia, precio_mes + precio_fijo_potencia

In [ ]:
df_precio_mes_or = pd.DataFrame.from_dict(mesuario)
df_precio_mes_or.index = ['Exceso Potencia\n(€/mes)', 'Potencia Fija\n(€/mes)', 'Total\n(€/mes)']
#Print 
print(tabulate(df_precio_mes_or,  headers="keys", showindex=True, tablefmt='fancy_grid'))

In [ ]:
#Formulas

#Datos del Problema
potC = [8, 10, 13, 14, 15, 15]
tarifa = '6_3TD'
tipo_punto = 3

def facturacion_EP(potC, tarifa, tipo_punto):

    termino_exceso_potencia = precio_exceso_potencia(tarifa, tipo_punto)
    periodos = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6']
    mesuario = {}

    if tipo_punto == 1 or 2 or 3:

        df_exceso_euro_mes = pd.DataFrame()

        for date in list_unique_dates:
            new_df = cups_n1[cups_n1["Fecha"].str.startswith(date)]
            new_df.reset_index(drop=True, inplace=True)
            list_exceso = []
            list_SinExceso = []

            for i, Pdj in enumerate(new_df.ActivaEntrante):
            
                num_PX = int(new_df.periodo[i][1:])
                Pcp = potC[(num_PX-1)]

                if Pdj > Pcp:
                    result_4y5 = 2 * (Pdj - Pcp) * termino_exceso_potencia
                elif Pdj < Pcp or Pdj == Pcp:
                    result_4y5 = 0

                list_exceso.append(result_4y5)

            df_exceso_euro_mes[date] = pd.Series(list_exceso)

        df_exceso_euro_mes = df_exceso_euro_mes.fillna(0)
        precio_mes = round(df_exceso_euro_mes.sum(axis='index'), 2)

        return precio_mes

    elif tipo_punto == 4 or 5:

        for date in list_unique_dates:
            list_mes = []

            for period in periodos:
                new_df = cups_n1[(cups_n1["Fecha"].str.startswith(date)) & (cups_n1['periodo'] == period)]
                new_df.reset_index(drop=True, inplace=True)
                list_exceso = []

                for i, Pdj in enumerate(new_df.ActivaEntrante):
                
                    num_PX = int(period[1:])
                    Pcp = potC[(num_PX-1)]

                    if Pdj > Pcp:
                        result_123 = (Pdj - Pcp) ** 2
                    elif Pdj < Pcp or Pdj == Pcp:
                        result_123 = 0

                    #list_SinExceso.append(result_123)
                    list_exceso.append(result_123)

                sumatorio_1 = sum(list_exceso)
                #1Formula
                fep = precio_coeficiente_Kp(tarifa, period) * termino_exceso_potencia * math.sqrt(sumatorio_1)
                list_mes.append(fep)

            mesuario[date] = round(int(sum(list_mes)), 2)
            
        return mesuario


# Función Objetivo (Spicy.Minimize)

In [ ]:
from scipy.optimize import minimize, rosen, rosen_der

In [ ]:
#Valores Iniciales
x0 = [2, 2, 3, 4, 4, 5]
tarifa = '6_3TD'
tipo_punto = 4

#sum_f0
precio_fijo_potencia = facturacion_potencia(tarifa, x0)
precio_exceso_potencia = facturacion_EP(tarifa, tipo_punto, x0)
precio_total = []

for i, exceso_potencia in enumerate(precio_exceso_potencia):
    price = exceso_potencia + precio_fijo_potencia
    precio_total.append(price)

sum_f0 =  sum(precio_total)

In [ ]:
sum_f0

In [ ]:
#Initialize Model   //{ #1) m = GEKKO() }
#2)
def objective(x):
    #x1 = x[0]
    #x2 = x[1]
    #x3 = x[2]
    #x4 = x[3]
    #x5 = x[4]
    #x6 = x[5]

    precio_fijo_potencia = facturacion_potencia(tarifa, x)
    precio_exceso_potencia = facturacion_EP(tarifa, tipo_punto, x)
    precio_total = []

    for i, exceso_potencia in enumerate(precio_exceso_potencia):
        price = exceso_potencia + precio_fijo_potencia
        precio_total.append(price)

    return sum(precio_total)

In [ ]:
'''RESTRICCIONES:
Diferenciamos entre tipo ecuación y tipo inecuación. Tendremos un total de 4 restricciones que corresponden a la obligatoriedad de que la combinación 
de potencias sea en orden ascendente o igual, es decir, potencias de periodos inferiores no pueden ser mayores a potencias de periodos superiores, como mínimo iguales.
La restricción 5 viene del conocimiento en mercado, ya que se obliga a que la potencia del último periodo sea igual a la que se tenía contratada porque este cambio de potencia 
suele acarrear costes importantes y el ratio beneficio de optimización/coste modificación suele ser muy pequeño.

                                                    def constraint5_ineq(x): 
                                                        return x[5] - x0[5] 
                                                    con4_ineq = {'type': 'ineq', 'fun': constraint4_ineq} 

Finalmente, las “dos” restricciones 6 sirven para obligar a que la función objetivo tiene que mejorar el coste total inicial, el cual usa de partida.
                '''
                
def constraint1_ineq(x): 
    return x[1] - x[0] 
con1_ineq = {'type': 'ineq', 'fun': constraint1_ineq}

def constraint2_ineq(x): 
    return x[2] - x[1] 
con2_ineq = {'type': 'ineq', 'fun': constraint2_ineq}

def constraint3_ineq(x): 
    return x[3] - x[2] 
con3_ineq = {'type': 'ineq', 'fun': constraint3_ineq} 

def constraint4_ineq(x): 
    return x[4] - x[3] 
con4_ineq = {'type': 'ineq', 'fun': constraint4_ineq}

def constraint5_ineq(x): 
    return x[5] - x[4] 
con5_ineq = {'type': 'ineq', 'fun': constraint5_ineq}

def constraint6_ineq(x): 
    return sum_f0 - objective(x) 
con6_ineq = {'type': 'ineq', 'fun': constraint6_ineq}

In [ ]:
# LIMITES 
b = (0.0, None) 
bnds = (b, b, b, b, b, b)

In [ ]:
#CONSTRAINS
cons = [con1_ineq, con2_ineq, con3_ineq, con4_ineq, con5_ineq, con6_ineq]

In [ ]:
#SOLUTION
sol = minimize(objective, x0, method='SLSQP', bounds= bnds, constraints= cons, tol=None, callback=None)

In [ ]:
print(sol)

In [ ]:
new_df = cups_n1[cups_n1['periodo'] == period]
new_df.reset_index(drop=True, inplace=True)
x = [2., 2., 3., 4., 4., 5.]
j = int(new_df.periodo[57][1:])
Pcp = x[(j-1)]
print(Pcp)
print(j)

In [ ]:
def facturacion_total(potC, tarifa, tipo_punto):
#termino_exceso_potencia = precio_exceso_potencia(tarifa, tipo_punto)
    periodos = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6']
    mesuario = {}
    list_exceso = []

    if np.any(tipo_punto <= 3):
        price = tbl_excPot_med123[tbl_excPot_med123['Tarifa'] == tarifa]['Precio_exceso_potencia_(€/kW*día)'].item()
        for i, Pdj in enumerate(cups_n1.ActivaEntrante):
            j = int(cups_n1.periodo[i][1:])
            Pcp = potC[(j-1)]
            if Pdj > Pcp:
                fep = 2 * (Pdj - Pcp) * price
            elif Pdj < Pcp or Pdj == Pcp:
                fep = 0
            list_exceso.append(fep)
        #Formula
        fep = sum(list_exceso)

    else:
        price = tbl_excPot_med4y5[tbl_excPot_med4y5['Tarifa'] == tarifa]['Precio_exceso_potencia_(€/kW*día)'].item()
        for period in periodos:
            coeficiente_Kp = precio_coeficiente_Kp(tarifa, period)
            new_df = cups_n1[cups_n1['periodo'] == period]
            new_df.reset_index(drop=True, inplace=True)
            sumatorio_1 = []
            for i, Pdj in enumerate(new_df.ActivaEntrante):
                j = int(new_df.periodo[i][1:])
                Pcp = potC[(j-1)]
                if Pdj > Pcp:
                    result_123 = (Pdj - Pcp) ** 2
                else:
                    result_123 = 0
                sumatorio_1.append(result_123)
            sumatorio_2 = sum(sumatorio_1)
            #Formula
            fep = coeficiente_Kp * price * math.sqrt(sumatorio_2)
            list_exceso.append(fep)
        fep = sum(list_exceso)

    return round(fep,2)


def objective(x):
    fep = facturacion_total(x, tarifa, tipo_punto)
    return fep


In [ ]:
fop = facturacion_total([8, 10, 13, 14, 15, 15], '6_3TD', 1)

In [ ]:
print(x)

In [ ]:

#GEKKO SOLUTION
from gekko import GEKKO    
m = GEKKO()
#help(m)

#Define parameters
eq = m.Param(value=potC[5])

#Variables
x1,x2,x3,x4,x5,x6 = [m.Var() for i in range(6)]

#Initial values
x1.value = potC[0]
x2.value = potC[1]
x3.value = potC[2]
x4.value = potC[3]
x5.value = potC[4]
x6.value = potC[5]

#lower bounds & upper bounds
x1.lower = 0
x2.lower = 0
x3.lower = 0
x4.lower = 0
x5.lower = 0
x6.lower = 0

# equation
m.Equation(x1 <= x2)
m.Equation(x2 <= x3)
m.Equation(x3 <= x4)
m.Equation(x4 <= x5)
m.Equation(x5 <= x6)
m.Equation(x6 == eq)

# Objective
m.Obj(facturacion_EP(tarifa,tipo_punto, [x1, x2, x3, x4, x5, x6]))

#Solve Simulation
m.options.IMODE = 3
m.solve(remote = True)

In [ ]:
#Results
print('')
print('Results:')
print('x1: ' + str(x1.value))
print('x2: ' + str(x1.value))
print('x3: ' + str(x1.value))
print('x4: ' + str(x1.value))
print('x5: ' + str(x1.value))

In [ ]:
#Contrains